Packages

In [ ]:
import nltk
import spacy
import pandas as pd
import re
from gensim.models.phrases import Phrases, Phraser
from collections import defaultdict
import multiprocessing
from gensim.models import Word2Vec

Import Data

In [ ]:
df = pd.read_json("./arxiv-metadata-oai-snapshot.json")
df.head()

Prepping Data

In [ ]:
df = df.dropna().reset_index(drop=True)
df = df.sort_values(by="year")
abstracts = df['summary'].str.lower()

Cleaning Data

In [ ]:
nlp = spacy.load('en', disable=['ner', 'parser'])

In [ ]:
def deep_clean(document):
    texts = [token.lemma_ for token in document if not token.is_stop]
    if len(texts) > 2:
        return ' '.join(texts)

In [ ]:
prep_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['summary'])
cleaned_text = [deep_clean(doc) for doc in nlp.pipe(prep_cleaning, batch_size=5000, n_threads=-1)]

In [ ]:
clean_df = pd.DataFrame({'clean': cleaned_text})
clean_df = clean_df.dropna().drop_duplicates()

Phrases

In [ ]:
process_phrases = [row.split() for row in clean_df['clean']]

In [ ]:
phrases = Phrases(process_phrases, min_count=30, progress_per=10000)
bigram = Phraser(phrases)

In [ ]:
sentences = bigram[process_phrases]

Word Frequencies

In [ ]:
frequencies = defaultdict(int)
for each in sentences:
    for i in each:
        frequencies[i] += 1
len(frequencies)
sorted(frequencies, key=frequencies.get, reverse=True)

Training

In [ ]:
cores = multiprocessing.cpu_count()

training_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-2)

In [ ]:
training_model.build_vocab(sentences, progress_per=10000)

In [ ]:
training_model.train(sentences, total_examples=training_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
training_model.init_sims(replace=True)

In [ ]:
training_model.wv.most_similar(positive=["software"])

In [ ]:
training_model.wv.similarity("technology", 'era')